# Machine Learning: A Primer for Radiologists

## Contents

* Machine Learning Basics
* Machine Learning Workflow
* Commonly Used Models
* Notes and Tips
* Applications in Radiology

### Machine Learning Basics
* Unsupervised and Supervised
* Bias and Variance
* Model Specification and Training
* Training and Testing
* Example: Linear Regression
  * Input and Output
  * Parameter Estimation
  * Training and Testing

#### Unsupervised and Supervised Learning

| Unsupervised | Supervised |
| :-------------: | :-------------:|
| Aims to "summarize" data  | Aims to "learn" a function between input data and output |
| Methods to reduce dimensionality of data | Methods to train a system for prediction |
| Clustering, Compression, Association Rules | Regression |

#### Unsupervised and Supervised Learning
<table style="width:100%">
  <tr>
    <td>These methods are not exclusive of each other and can compliment each other!</td>
    <td rowspan="2"><img src="assets/images/gene_expression_regression.jpg" alt='gene_express'/></td>
  </tr>
  <tr>
    <td>Example: reducing gene expression data to summarize the most important gene expression patterns of a dataset (unsupervised), which is then used to estimate survival (supervised).</td>
  </tr>
</table>

```python
def yeah():
    return True
```